# Assignment 2

Complete the code snippets in a colab notebook. 
Download the notebook file and submit in Blackboard.

In [45]:
import numpy as np

In [46]:
class Call:
    
    def __init__(self, strike):
        self.strike = strike

    def __call__(self, spot):
        return np.maximum(spot - self.strike, 0.)

In [47]:
class VanillaOption:
    
    def __init__(self, maturity, payoff):
        self.maturity = maturity
        self.payoff = payoff

In [48]:
class BlackScholesProcess:

    def __init__(self, spot, rate, volatility):    
        self.spot = spot
        self.rate = rate
        self.volatility = volatility
                 
    def evolve(self, time, norms):
        S_T = self.spot * np.exp((self.rate-0.5 * self.volatility ** 2) * time + self.volatility * (np.sqrt(time) * norms))
        return S_T

In [49]:
class FixedRateYieldCurve:

    def __init__(self, rate): 
        self.rate = rate

    def discount(self, maturity ):
        return np.exp (-self.rate * maturity)

Here is a function that can use your classes:

In [50]:
def mc_price_european(process, option, yield_curve, n_paths):
    std_norms = np.random.normal(size=n_paths)
    S_T = process.evolve(option.maturity, std_norms)
    V_T = yield_curve.discount(option.maturity) * option.payoff(S_T)
    V_0 = np.average(V_T)   

    return V_0

Some example inputs:

In [51]:
T = 5.
S_0 = 100.
vol = 0.2
r = 0.02
K = 100
K2 = 120
N = 10000

Let's instantiate objects from your classes:

In [52]:
yield_curve = FixedRateYieldCurve(r)
rand_process = BlackScholesProcess(S_0, r, vol)
call_payoff = Call(K)
call_option = VanillaOption(T, call_payoff)

and use them:

In [53]:
c = mc_price_european(rand_process, call_option, yield_curve, N)
print(c)


22.367740171962637


Now, do the same with a couple more payoff types:

In [54]:
class Put:

    def __init__(self, strike):
        self.strike = strike

    def __call__(self, spot):
        return np.maximum(self.strike - spot, 0.)

In [55]:
class DoubleDigital:
    def __init__(self, K, K2):
        self.K = K
        self.K2 = K2

    def __call__(self, spot):
        x = np.array(spot) 
        z = np.zeros(len(x))
        
        for i in range(len(x)):
            if self.K< x[i] < self.K2 :
                z[i] = 1
        return np.array(z)

and use them:

In [44]:
put_payoff = Put(K)
put_option = VanillaOption(T, put_payoff)
put_price = mc_price_european(rand_process, put_option, yield_curve, N)
print(put_price)

12.476975661584747


In [56]:
dbl_digit_payoff = DoubleDigital(K, K2)
double_digital_option = VanillaOption(T, dbl_digit_payoff)
dbl_digital_price = mc_price_european(rand_process, double_digital_option, yield_curve, N)
print(dbl_digital_price)

0.14097366973000247
